# Recommendation

dado um usuario e item no grafo, predizer se vai ser recomendação ou nao

In [ ]:
!gdown 1XFiH0-J1r9DepyhfAzQUCe8pIClddNHw
!gdown 1U-qJ0Aayp2srzlIxiztEpTjIolp9fWya
!gdown 163rDALChJadKYDgyQIDoXPxHzaKzR0gR

Downloading...
From: https://drive.google.com/uc?id=1XFiH0-J1r9DepyhfAzQUCe8pIClddNHw
To: /content/df_interest.pkl
100% 624k/624k [00:00<00:00, 141MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U-qJ0Aayp2srzlIxiztEpTjIolp9fWya
To: /content/df_outlier.pkl
100% 171k/171k [00:00<00:00, 94.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=163rDALChJadKYDgyQIDoXPxHzaKzR0gR
To: /content/movies_information_with_embeddings.pkl
100% 47.7M/47.7M [00:00<00:00, 70.1MB/s]


In [ ]:
import pandas as pd

df_int = pd.read_pickle('df_interest.pkl')
df_out = pd.read_pickle('df_outlier.pkl')
df_movies = pd.read_pickle('movies_information_with_embeddings.pkl')

In [ ]:
!pip install networkx==2.6

In [ ]:
from sklearn.model_selection import KFold
import networkx as nx

def generate_graph_enriched(df_train, df_movies):
  G = nx.Graph()

  for _,row in df_train.iterrows():
    user = str(row['user']) + ':user'
    item = str(row['item']) + ':item'

    keywords = df_movies[df_movies.movie_id == int(row['item'])].keywords.iloc[0]  
    genres = df_movies[df_movies.movie_id == int(row['item'])].genres.iloc[0]
    review = df_movies[df_movies.movie_id == int(row['item'])].reviews.iloc[0]

    G.add_edge(user,item)
    G.add_edge(item, str(row['item'])+':review')
    for keyword in keywords:
      G.add_edge(item,keyword['name'] + ':keyword')
    
    for genre in genres:
      G.add_edge(item,genre['name'] + ':genre')

    
  for node in G.nodes:
    if 'item' in node:
      G.nodes[node]['features'] = df_movies[df_movies.movie_id == float(node.split(':')[0])].overview_embedding.iloc[0]
      G.nodes[node]['type'] = 'item'
    elif 'review' in node:
      G.nodes[node]['features'] = df_movies[df_movies.movie_id == float(node.split(':')[0])].review_embedding.iloc[0]
      G.nodes[node]['type'] = 'review'
    elif 'genre' in node:
      G.nodes[node]['type'] = 'genre'
    elif 'user' in node:
      G.nodes[node]['type'] = 'user'
    elif 'keyword' in node:
      G.nodes[node]['type'] = 'keyword'

  return G


def train_test_split_ocl_recommendation(kf, df_int):
    train_test = []

    for train_index, test_index in kf.split(df_int):
        train_test.append((df_int.iloc[train_index], df_int.iloc[test_index]))
    
    return train_test

def foldValidation(folds):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    return kf

folds = 5
kf = foldValidation(folds)
train_test = train_test_split_ocl_recommendation(kf, df_int)

G_1 = generate_graph_enriched(train_test[0][0], df_movies)
G_2 = generate_graph_enriched(train_test[1][0], df_movies)
G_3 = generate_graph_enriched(train_test[2][0], df_movies)
G_4 = generate_graph_enriched(train_test[3][0], df_movies)
G_5 = generate_graph_enriched(train_test[4][0], df_movies)

In [ ]:
(len(G_1.nodes()) + len(G_2.nodes()) + len(G_3.nodes()) + len(G_4.nodes())+ len(G_5.nodes()))/5

8773.6

In [ ]:
(len(G_1.edges()) + len(G_2.edges()) + len(G_3.edges()) + len(G_4.edges())+ len(G_5.edges()))/5

30471.0

In [ ]:
count = 0
for g in [G_1.nodes(), G_2.nodes(),G_3.nodes(),G_4.nodes(),G_5.nodes()]:
  for node in g:
    if 'item' in node:
      count+=1
    elif 'review' in node:
      count+=1

count/5

2396.8

# Event

In [ ]:
!gdown 1IwsJpSuf11mH4o-nxhAa_79M3h1Reqti

Downloading...
From: https://drive.google.com/uc?id=1IwsJpSuf11mH4o-nxhAa_79M3h1Reqti
To: /content/run_10_gold_standard_5w1h_graph_hin.nx
100% 662k/662k [00:00<00:00, 121MB/s]


In [ ]:
import networkx as nx
G_ev = nx.read_gpickle('run_10_gold_standard_5w1h_graph_hin.nx')

In [ ]:
len(G_ev.nodes())

579

In [ ]:
len(G_ev.edges())

803

In [ ]:
for n in G_ev.neighbors('20000311: online media:iptc_code'):
  print(n)

883e31031f9266f4e6a62aceb8807e8fd447306d20850d9f417238cb:event
58316513017cff90c5cf66daade95beced3919684ef93b4206278c82:event
4f38a182b733b32f105f00b1d3354cd4568b867b22991cf7fd89046c:event
3819549c5ea2cdd5d32008086adb0722d690d898fb000ac4d8573904:event
aaa7a68a35731e6cbf4413db90826fc59022dc90512dd90b1ceca96e:event
e0ff5adbdbd19e4bb271818149bbf8e063bf8a52b8429f8af48599cd:event


In [ ]:
dic = {}
for node in G_ev.nodes():
  if 'event' in node:
    l = G_ev.nodes[node]['label']
    if l not in dic:
      dic[l] = 1
    else:
      dic[l]+=1
dic

{'toberone-gate': 11,
 'clinton blames comey': 11,
 'china well': 4,
 'cubs win championship': 10,
 'seattle shooting': 3,
 'gaga protests': 5,
 'f1': 12,
 'clinton clear': 9,
 'consulate attack': 7,
 'trump and obama': 9,
 'tram': 10,
 'tram victims': 4,
 nan: 1}

In [ ]:
G_ev.nodes()

NodeView(('ce21d3b18b0f22a3c44dab7f8e22e70692ae814e48bd5afc6c94a6f5:event', 'is increased to reduce the amount:what', 'November 8 , 2016:when', 'Poundland:where', 'gap between triangles:who', 'in new window ) Click to share on Facebook (:how', '3:cluster_code', '20000556: cafe:iptc_code', '20000558: restaurant:iptc_code', ':iptc_code', '13a20be02e984533782dc2c6cd2f468b879ce4be88127566da043b0f:event', 'sparks outrage after changing iconic shape:what', 'UK:where', 'Toblerone:who', 'the distinctive triangles of its bars .:how', '30:cluster_code', 'f326929913960e17a3afe71085002a532beaca6d68c0770372a1832d:event', 'Blames F.B.I. Director:what', 'the last minute:when', 'Nevada:where', 'Hillary Clinton:who', 'because President Obama had vowed to bestow " gifts " on Democratic special interests groups , namely African-Americans , Hispanics and young people .:why', 'In particular , white suburban women who had been on:how', '25:cluster_code', '7b1f91bcb6dc19b7db1e86930b69f5e65ff5984db394440468f7

In [ ]:
count = 0
for node in G_ev:
  if 'event' in node:
    count+=1

count

96

# Music


In [ ]:
import networkx as nx

!gdown 1g9vgU2eoWV8Lnw6AYjOuoK-f2PU-7U47
!gdown 1KWmBd9nN1BNAEmBbqUjBCYEqsIGvaxzv

Downloading...
From: https://drive.google.com/uc?id=1g9vgU2eoWV8Lnw6AYjOuoK-f2PU-7U47
To: /content/graph_filtered.pkl
100% 5.07M/5.07M [00:00<00:00, 120MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KWmBd9nN1BNAEmBbqUjBCYEqsIGvaxzv
To: /content/songs_normalize_lyrics3.tsv
100% 4.34M/4.34M [00:00<00:00, 227MB/s]


In [ ]:
!pip3 install pickle5
import pickle5 as pickle

with open('graph_filtered.pkl', "rb") as fh:
  G_mu = pickle.load(fh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256401 sha256=3270967d9fefad1f6a517fd23a2d6e4f41a270e4b6575e75ffca53d475c82292
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
len(G_mu.nodes())

2125

In [ ]:
len(G_mu.edges())

5243

In [ ]:
count = 0
for node in G_mu:
  if 'song' in node:
    count+=1

count

1529

In [ ]:
import pandas as pd

df = pd.read_csv('songs_normalize_lyrics3.tsv', sep='\t')

In [ ]:
for node in G_mu.nodes():
  if 'song' in node:
    if int(df[df['song'] == node.split(':')[1]]['popularity'].iloc[0]) > 50:
      G_mu.nodes[node]['label'] = 'hit'
    else:
      G_mu.nodes[node]['label'] = 'flop'

In [ ]:
G_mu.nodes()['song:Wings']

{'features': array([-1.52126431e-01,  1.92236319e-01, -1.57880262e-02,  2.05808833e-01,
        -2.96475440e-01, -1.16043724e-01,  4.46270257e-01,  3.39406729e-01,
         4.41738844e-01, -3.83401513e-01, -3.94024670e-01, -1.05127394e+00,
        -2.23717064e-01, -5.02886891e-01, -3.25048685e-01,  2.66067952e-01,
         9.44321573e-01, -1.48720369e-01,  4.07440722e-01,  7.45550275e-01,
        -1.71388865e-01,  3.84072125e-01,  3.87645483e-01, -1.07474916e-01,
         3.56991947e-01, -5.01661181e-01,  1.37993237e-02, -1.16911769e-01,
         1.08085349e-01, -6.05422914e-01,  1.78958297e-01,  2.81632900e-01,
        -2.26496667e-01, -1.18480945e+00,  7.31811374e-02, -3.01891994e-02,
        -3.47268403e-01, -6.21421337e-01, -1.16831303e-01, -7.09319174e-01,
        -2.13817909e-01, -2.98644174e-02, -5.83836198e-01, -4.06693280e-01,
        -8.55724454e-01, -3.14301550e-01,  3.81521881e-01, -2.35388398e-01,
        -5.62677622e-01, -5.56283183e-02, -1.56792641e-01,  1.61848485e-01,


# FakeNews


In [ ]:
!pip install git+https://github.com/GoloMarcos/FKTC/
from FakeNewsTextCollections import datasets
datasets_dictionary = datasets.load()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/GoloMarcos/FKTC/ to /tmp/pip-req-build-lty1qjdv
  Running command git clone --filter=blob:none --quiet https://github.com/GoloMarcos/FKTC/ /tmp/pip-req-build-lty1qjdv
  Resolved https://github.com/GoloMarcos/FKTC/ to commit b8c349e52a7f9264832007e85fa4df67c2cab7cb
  Preparing metadata (setup.py) ... done
  Created wheel for FakeNewsTextCollections: filename=FakeNewsTextCollections-1.1.0-py3-none-any.whl size=2130 sha256=b51d5e45adad8ea560ba7890e28483567549803f8c37dd28336f6cb85bdfef18
  Stored in directory: /tmp/pip-ephem-wheel-cache-cg0pkqog/wheels/a5/65/0c/83285d856a9e55446d73fb609d86dfb16a8d1cb854aaeaaa2b
Successfully built FakeNewsTextCollections
Download News.zip
Download Finished!
Unzip News.zip
Unzip Finished!


In [ ]:
df = datasets_dictionary['fcn']

In [ ]:
df

,index,text,class,fold,features,features_normalized,DistilBERT Multilingua,BERT,DistilBERT,RoBERTa
0,NoticiasColetas/noticiasFakes/noticia_fake_287...,""" #Verificamos: Atentado contra Jair Bolsonaro...",1,0,"[432.0, 210.0, 37.0, 131.0, 96.0, 40.0, 63.0, ...","[0.1123273390669794, 0.10434348477284076, 0.34...","[-0.02726702, -0.054255705, 0.035042983, -0.03...","[0.02928251, 0.27480382, 0.895543, 0.7502444, ...","[-0.6702545, 1.1699916, 0.20820662, -0.2359720...","[0.71498674, 0.6416639, -0.11766751, 0.706066,..."
1,NoticiasColetas/noticiasFakes/noticia_fake_403...,"""""""É #FAKE que Cármen Lúcia disse após a eleiç...",1,1,"[129.0, 42.0, 8.0, 67.0, 33.0, 9.0, 20.0, 16.0...","[0.03335939536095908, 0.020469296055916127, 0....","[0.01982586, -0.02503765, 0.04633246, -0.03950...","[0.27488157, -0.37114394, 0.81004906, 0.461189...","[-0.8606741, 0.7280851, -0.3952505, -0.6577937...","[1.4546995, 0.84561014, 0.50348055, 0.6964588,..."
2,NoticiasColetas/noticiasFakes/noticia_fake_857...,"""Checamos o balanço do primeiro ano de Temer n...",1,2,"[463.0, 259.0, 43.0, 131.0, 79.0, 53.0, 45.0, ...","[0.12040656763096168, 0.1288067898152771, 0.39...","[-0.027151074, 0.016133722, 0.030654192, -0.04...","[0.16805542, -0.6324302, 0.68457615, -0.122011...","[0.0487945, 0.6889197, 0.37327543, -0.49556065...","[-0.24594194, -0.03368065, -0.416129, -0.68468..."
3,NoticiasColetas/noticiasFakes/noticia_fake_892...,"""""""É falso que governo federal cortará verbas ...",1,3,"[107.0, 47.0, 5.0, 43.0, 25.0, 15.0, 13.0, 16....","[0.02762574928329424, 0.02296555167249126, 0.0...","[-0.026940623, 0.0015602746, 0.0072067054, -0....","[0.45953202, -0.3037919, 0.72601396, -0.054937...","[-1.2321968, 0.54112625, -0.028838016, -0.0941...","[1.098731, 0.6180506, 0.42614856, 1.1549015, 0..."
4,NoticiasColetas/noticiasFakes/noticia_fake_942...,É #FAKE que vídeo com calçadão de Bangu alagad...,1,4,"[79.0, 38.0, 10.0, 30.0, 21.0, 3.0, 9.0, 11.0,...","[0.020328381548084442, 0.018472291562656017, 0...","[-0.009362391, -0.026783256, -0.03109894, -0.0...","[-0.030948788, 0.23847307, 1.2021756, 0.325094...","[-1.1616026, 0.8064826, -0.29577702, -0.174658...","[0.03776347, 0.5396594, 0.85794723, -0.1394677..."
...,...,...,...,...,...,...,...,...,...,...
2059,NoticiasColetas/noticiasFatos/noticia_real_620...,"""""""Defesa de Cunha pede suspensão de prazo das...",-1,-1,"[201.0, 102.0, 6.0, 88.0, 36.0, 49.0, 15.0, 28...","[0.05212405525149857, 0.05042436345481778, 0.0...","[0.006411521, -0.0038219988, 0.037522394, -0.0...","[-0.2048057, -0.48409197, 1.4513916, 0.0150171...","[-0.65570337, 0.5656601, -0.5492626, -0.379793...","[0.38845313, 0.4743366, 0.07403394, 0.12973994..."
2060,NoticiasColetas/noticiasFatos/noticia_real_682...,"""""""Militares já fizeram sacrifício, mas podem ...",-1,-1,"[227.0, 86.0, 17.0, 79.0, 67.0, 24.0, 42.0, 41...","[0.05890018243419338, 0.04243634548177734, 0.1...","[0.01844728, -0.0059580733, 0.0061615775, -0.0...","[0.25521225, -0.7519288, 0.7098324, -0.0981794...","[-0.33205926, 0.51175046, 0.18040699, -0.38288...","[1.1151724, 0.72488904, 0.17918283, 0.5692972,..."
2061,NoticiasColetas/noticiasFatos/noticia_real_597...,"""""""Bolsonaro diz que Bebianno terá de voltar à...",-1,-1,"[134.0, 55.0, 10.0, 60.0, 47.0, 26.0, 23.0, 19...","[0.03466249674224655, 0.026959560659011485, 0....","[0.0015588008, -0.0329165, 0.0708499, -0.04436...","[0.32660016, -0.080995776, 0.8348194, 0.203966...","[-0.47287697, 0.49990296, -0.2112068, -0.23997...","[0.45177132, 0.53245926, 0.95159024, 0.9282363..."
2062,NoticiasColetas/noticiasFatos/noticia_real_3.txt,""" #Verificamos: Laurita Vaz negou habeas corpu...",-1,-1,"[183.0, 100.0, 10.0, 64.0, 43.0, 15.0, 17.0, 2...","[0.047432890278863694, 0.04942586120818772, 0....","[0.028915882, -0.0066966065, -0.014331256, -0....","[-0.120263785, -0.6095586, 0.7294569, 0.228145...","[-0.52274156, 0.59909266, -0.34061524, -0.4261...","[0.91650283, -0.0079382025, 0.17729926, 0.3378..."


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer

def remove_stopwords(text,stop_words):
  
  # tudo para caixa baixa
  s = str(text).lower() 

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

def stemming(tokens,stemmer):
  tokens_stems = [stemmer.stem(word) for word in tokens]
  return tokens_stems

def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('portuguese'), stemmer=nltk.stem.RSLPStemmer()):
  tokens = remove_stopwords(doc,stop_words)
  return stemming(tokens,stemmer)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [ ]:
VSM = CountVectorizer(tokenizer=meu_tokenizador, min_df=2, ngram_range=(1,1))
X = VSM.fit_transform(df['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
VSM.get_feature_names_out()[:153]

array(['02h00twitterfacebookemailwhatsapph',
       '02h30twitterfacebookemailwhatsapp',
       '06h00twitterfacebookemailwhatsapp', '10h', '10h10', '10h30',
       '10h33', '10h40', '10h44', '10ª', '11h', '11h30', '11h45', '11h48',
       '11º', '12h', '12h30', '12h40', '12ª', '12º', '13h', '13h04',
       '13h30', '13ª', '13º', '14h', '14h00twitterfacebookemailwhatsapp',
       '14h10', '14h30', '14h30twitterfacebookemailwhatsapp', '14h40',
       '14h50', '14h55', '15h', '15h00twitterfacebookemailwhatsapp',
       '15h05twitterfacebookemailwhatsapp',
       '15h15twitterfacebookemailwhatsapp', '15h30',
       '15h33twitterfacebookemailwhatsappé', '15h36', '15h40', '15º',
       '16h', '16h20', '16h20twitterfacebookemailwhatsapp', '16h30',
       '16h35', '16h40', '17h', '17h00twitterfacebookemailwhatsapp',
       '17h15', '17h30', '17h30twitterfacebookemailwhatsapp', '17h40',
       '17º', '18h', '18h02', '18h07',
       '18h10twitterfacebookemailwhatsapp',
       '18h11twitterfaceb

In [ ]:
bag_of_word = pd.DataFrame(X.toarray(), columns=VSM.get_feature_names_out())
bag_of_word = bag_of_word[VSM.get_feature_names_out()[153:]]
bag_of_word 

,abajul,abal,abandon,abarc,abastec,abat,abc,abd,abdom,abdômen,...,órg,ótic,ótim,ô,ônibu,últ,únic,útel,úter,útil
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
B = nx.Graph()

B.add_nodes_from(bag_of_word.index.to_list(), bipartite='doc')

B.add_nodes_from(bag_of_word.columns, bipartite='term')

for index in bag_of_word.index.to_list():
  for c in bag_of_word.columns:
    if bag_of_word.loc[index][c] > 0:
      B.add_edge(index, c)

In [ ]:
for node in B.nodes():
  if B.nodes[node]['bipartite'] == 'doc':
    if df.loc[node]['class'] == 1:
      B.nodes[node]['label'] = 'fake'
      B.nodes[node]['features'] = df.loc[node]['DistilBERT Multilingua']
    else:
      B.nodes[node]['label'] = 'real'
      B.nodes[node]['features'] = df.loc[node]['DistilBERT Multilingua']

In [ ]:
len(B.nodes())

10348

In [ ]:
len(B.edges())

318411

In [ ]:
B.nodes()

In [ ]:
count = 0
for node in B.nodes():
  print('node')
    count+=1

count

#Save Graphs

In [ ]:
from google.colab import drive

drive.mount("/content/drive/", force_remount = True)

Mounted at /content/drive/


In [ ]:
PATH_TO_PICKLE_DIR = "/content/drive/MyDrive/Multimodal_Fusion/grafos"

In [ ]:
import pickle

with open(f"{PATH_TO_PICKLE_DIR}/rec_sys_1.pkl", "wb") as file:
    pickle.dump(G_1, file)

with open(f"{PATH_TO_PICKLE_DIR}/rec_sys_2.pkl", "wb") as file:
    pickle.dump(G_2, file)

with open(f"{PATH_TO_PICKLE_DIR}/rec_sys_3.pkl", "wb") as file:
    pickle.dump(G_3, file)

with open(f"{PATH_TO_PICKLE_DIR}/rec_sys_4.pkl", "wb") as file:
    pickle.dump(G_4, file)

with open(f"{PATH_TO_PICKLE_DIR}/rec_sys_5.pkl", "wb") as file:
    pickle.dump(G_5, file)

In [ ]:
with open(f"{PATH_TO_PICKLE_DIR}/event.pkl", "wb") as file:
    pickle.dump(G_ev, file)

In [ ]:
with open(f"{PATH_TO_PICKLE_DIR}/music.pkl", "wb") as file:
    pickle.dump(G_mu, file)

In [ ]:
with open(f"{PATH_TO_PICKLE_DIR}/fakenews.pkl", "wb") as file:
    pickle.dump(B, file)